# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re

from tabula import read_pdf
from tabulate import tabulate
import pdfplumber

import openpyxl

from Script import ExtractPDFTables, MapLinks2Excel
import camelot.io as camelot
from camelot import plot


In [ ]:
from distributed import Client
client = Client()

In [ ]:
client.close()

In [ ]:
from ghostscript import Ghostscript



# 1 .Table Extraction and Mapping (SGD-GRI 2016 Link)

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/SDG-GRI/sdg-gri.pdf'
page_range = [list(range(3, 73)), list(range(74, 99))]
area = [80.51, 90.42, 561.96, 814.18]

tableTCFD_GRI = ExtractPDFTables(pdf_path, page_range, area)
df = tableTCFD_GRI.getTablesSDG_GRI('ESG-Frameworks/ESG Data/SDG_info.csv')

In [ ]:
df

## Mapping Collected Dataframe (SDG-GRI)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "SDG's"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapSDG_GRI()

## Mapping Collected Dataframe (GRI-SDG)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapGRI_SDG()

# 2. Table Extraction and Mapping (COH4B-GRI 2016 Link)

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI-COH4B/gri-coh4b.pdf'
page_range = list(range(13, 22))


tableGRI_COH4B = ExtractPDFTables(pdf_path, page_range)
df = tableGRI_COH4B.getTablesGRI_COH4B()
df = tableGRI_COH4B.setHeaders(df, 0)
df = tableGRI_COH4B.headerSwap(df, 'A. COHBP & \ndefinition', None, 'id')
df = tableGRI_COH4B.addDot(df, 'id')



df = tableGRI_COH4B.extractDisclosures1(df, 'D. GRI disclosures', 'GRI Standards', '[0-9]{3}-[0-9]{2}|[0-9]{3}-[0-9]{1}', re.search)



In [ ]:
# df

## Mapping Collected Dataframe (COH4B-GRI)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "COH4B"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapCOH4B_GRI()

## Mapping Collected Dataframe (GRI-COH4B)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"

mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.MapGRI_COH4B()

# 3. Table Extraction and Mapping (TCFD-GRI 2016 Link)

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/TCFD-GRI/TCFD to GRI.pdf'
page_range = list(range(49, 59))


tableTCFD_GRI = ExtractPDFTables(pdf_path, page_range)
df = tableTCFD_GRI.getTablesTCFD_GRI()
df = tableTCFD_GRI.extractDisclosures2(df, 'Related \ncode/\nparagraph', 'GRI Standards', r'[0-9]{3}-[0-9]{2}|[0-9]{3}-[0-9]{1}', re.findall)
df = tableTCFD_GRI.extractDisclosures2(df, 'Recommended \nDisclosures \n(TCFD Framework)', 'id', r'\w+\s[abc]\.', re.findall)




# [a-zA-Z]\.

## Mapping Collected Dataframe (TCDF-GRI)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "TCFD"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapTCFD_GRI()


## Mapping Collected Dataframe (GRI-TCDF)

In [ ]:
df = tableTCFD_GRI.df_gri_tcdf(df)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapGRI_TCFD()

# 4. Table Extraction and Mapping (GRI 2016- GRI 2021)

### Reading extracted csv from excel and fixing the columns names, this new csv file will be use for future mapping

In [ ]:
# df = pd.read_csv('ESG-Frameworks/Mapping-Standards/GRI 2016-2021//GRI2016-2021.csv', sep=';')
# df = df.iloc[0:67]
# df.rename(columns = {"GRI Standard Title": "GRI Standard Title 2016",
#                      "Disclosure Number": "Disclosure Number 2016",
#                      "Disclosure Name\nIndividual requirements ('a', 'b', 'c', etc.) are not listed here": 'GRI 2016 Description',
#                      "Section": "Section 2016",
#                      "Page Number": "Page Number 2016",
#                      "GRI Standard Title.1": "GRI 2021 Standard Title",
#                      "Disclosure Number.1": "Disclosure Number 2021",
#                      "Disclosure Name\nIndividual requirements ('a', 'b', 'c', etc.) are not listed here.1": 'GRI 2021 Description',
#                      "Section.1": "Section 2021",
#                      "Page Number.1": 'Page Number 2021'},
#                       inplace = True)


In [ ]:
# df.to_csv('GRI2016-2021.csv')

## Mapping GRI 2021 ON GRI 2016 sheet

#### Source code for method

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

wb = openpyxl.load_workbook(path_file)
ws = wb[sheet]

rows = ws.max_row

print(rows)
# print(rows)
for i in range(3, rows):
    if ws.cell(row=i, column=2).value != None:
        target_cell = ws.cell(row=i, column=2).value
        # print(target_cell)
        
        if target_cell != None:
            try:
                disclosure_to_add = df.loc[df['Disclosure Number 2016'] == target_cell]['Disclosure Number 2021'].item()
                ws.cell(row=i, column=4, value=str(disclosure_to_add))
                
                section_to_add = df.loc[df['Disclosure Number 2016'] == target_cell]['Section 2021'].item()
                	
                ws.cell(row=i, column=5, value=str(section_to_add))	

            except:
                pass
        
        
       

wb.save(path_file)

print(f'{sheet} sheet from Excel file have bee mapped')

In [ ]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/GRI 2016-2021//GRI2016-2021.csv')
df = df.drop('Unnamed: 0', axis=1)

### Mapping GRI 2021 to 2016 sheet using class method

In [ ]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/GRI 2016-2021//GRI2016-2021.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2016"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapGRI2016_2021()

## Mapping GRI 2016 ON GRI 2021sheet

In [ ]:
df = pd.read_csv('ESG-Frameworks/Mapping-Standards/GRI 2016-2021//GRI2016-2021.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
# excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
# sheet = "GRI 2021"

# a = []
# wb = openpyxl.load_workbook(excelFilePath)
# ws = wb[sheet]
# rows = ws.max_row

# regex = '[1-9]+-[1-9]+'

# for i in range(3, rows):
# 	if ws.cell(row=i, column=2).value == None:
# 		pass
    
# 	else:
# 		target_cell = ws.cell(row=i, column=2).value

# 		if target_cell != None:
# 			try:
# 				disclosure_to_add = df.loc[df['Disclosure Number 2021'] == target_cell]['Disclosure Number 2016'].values
# 				# print(disclosure_to_add)
# 				ws.cell(row=i, column=4, value='\n'.join(disclosure_to_add))

                
# 				section_to_add = df.loc[df['Disclosure Number 2021'] == target_cell]['Section 2016'].values

                
# 				ws.cell(row=i, column=5, value=section_to_add[0])

# 			except:
# 					pass

# wb.save(excelFilePath)

# # print(f"{sheet} sheet from Excel file have bee mapped with it's GRI 2021 equivalent")



In [ ]:
excelFilePath = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = "GRI 2021"


mapp = MapLinks2Excel(df, sheet, excelFilePath)
mapp.mapGRI2021_2016()

# 5 Table Extraction and Mapping(CASS CSR 4.0 - GRI 2016)

## Source code to extract pdf

In [ ]:
file_path = 'ESG-Frameworks/Mapping-Standards/CASSCSR-GRI/CASS_CSR-GRI.pdf'
page_range = list(range(7, 30))

pdf = pdfplumber.open(file_path, pages=page_range)
frames = []

for i in range(0, len(page_range)):
	try:
		page = pdf.pages[i]
		table = page.extract_table()
		frames.append(pd.DataFrame(table))
			
	except:
		pass

df =  pd.concat(frames)
df.columns = df.iloc[0]
df = df[1:]
find_duplicate = df['KPI \n(CASSCSR-4.0)'].isin(['KPI \n(CASSCSR-4.0)'])

df = df[~find_duplicate]

## Getting data from pdf CASS_GRI 2016

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/CASSCSR-GRI/CASS_CSR-GRI.pdf'
page_range = list(range(7, 30))


tableCASS_GRI = ExtractPDFTables(pdf_path, page_range)
df = tableCASS_GRI.getTablesCASS_GRI()

## Mapping to excel! source code  CASS ON GRI2016 SHEET

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'

sheet = 'GRI 2016'

regex = '\w[+-]?[0-9]+\.+\d+'

wb = openpyxl.load_workbook(path_file)
ws = wb[sheet]
rows = ws.max_row


for i in range(3, rows):
	if ws.cell(row=i, column=2).value == None:
		pass

	else:
		target_cell = ws.cell(row=i, column=2).value
		# print(target_cell)

        
		if target_cell != None:
            
			try:
				disclosure_to_add = df.loc[df['ID_GRI'] == target_cell]['ID_CASS_CSR'].values
				
                
				if len(disclosure_to_add) > 0:
					disclosure_to_add_ = [i[re.search(regex, i).start():re.search(regex, i).end()] for i in disclosure_to_add]
					# print(disclosure_to_add_)
					ws.cell(row=i, column=12, value='\n'.join(disclosure_to_add_))
                    
				section_to_add = df.loc[df['ID_GRI'] == target_cell]['KPI \n(CASSCSR-4.0)'].values
                
                
				if len(section_to_add)>0:
                    
					# print(section_to_add)
					ws.cell(row=i, column=13, value='\n'.join(section_to_add))	

			except:
				pass
wb.save(path_file)

## Mapping to Excel Class method CASS on GRI 20016

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'

sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapCASS_GRI1016()

## Mapping to Excel GRI2016 ON CASS 

### Source Code

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'

sheet = 'CASS CSR 4.0'


wb = openpyxl.load_workbook(path_file)
ws = wb[sheet]
rows = ws.max_row



    
    
for i in range(3, len(df['ID_CASS_CSR'])):

    ws.cell(row=i, column=1, value=df['ID_CASS_CSR'][i])
    ws.cell(row=i, column=2, value=df['KPI \n(CASSCSR-4.0)'][i])
    ws.cell(row=i, column=3, value=df['ID_GRI'][i])
    ws.cell(row=i, column=4, value=df['Disclosure title (GRI)'][i])
    

wb.save(path_file)


### Mapping GRI TO CASS WITH CLASS METHOD

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'

sheet = 'CASS CSR 4.0'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI2016_CASS()

In [ ]:
df

# 6. Table Extraction and Mapping(CDP 2017 - GRI 2016)

## Getting Tables from doc

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/CDP17-GRI/Cdp 2017 to GRI.pdf'
page_range = list(range(2, 32))


tableCDP_GRI = ExtractPDFTables(pdf_path, page_range)
df = tableCDP_GRI.getTablesCDP_GRI()

## Mapping CDP 2017 to GRI

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapCDPCC17_GRI()


## Mapping GRI to CDP 2017

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'

sheet = 'CDP 2017'

wb = openpyxl.load_workbook(path_file)
ws = wb[sheet]

# print(df['CDP'])
regex = '[a-zA-Z]+.+\d'

r = 3
for i in df['CDP']:
    if re.search(regex, i):
#         print('--------------------------------------')
        ws.cell(row=r, column=1, value=i)
#         print(i)
        r+=1


regex_2 = 'GRI \w+\d:'
    
r = 3
for i in df['GRI']:
    if i != None and re.search(regex_2, i):
        ws.cell(row=r, column=2, value=i)
        r+=1

        

wb.save(path_file)


	

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'CDP CC 2017'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI_CDPCC17()


# 7. Table Extraction and Mapping (CDP Water 2018 - GRI)

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/CDP Water 2018 - GRI/CDP 2018 Water to GRI.pdf'
page_range = list(range(6, 29))


tableCDP_GRI = ExtractPDFTables(pdf_path, page_range)
df = tableCDP_GRI.getTablesCDP_GRI()




## Mapping CDP Water 2018 on GRI 2016 sheet

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapCDP18_GRI()


## Mapping GRI 2016 ON CDP Water 2018 sheet

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'CDP WATER 2018'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI_CDP18()


In [ ]:
df['GRI Disclosures'].str.extract(r'(?m)(?:\r\n?|^)\d\d\d-\d\d:\s*(.*\S)\s*$')

# 9. Table Extraction and Mapping CDP CC 20XX - TCFD

Note: CC means climate change

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/CDP - TCFD/CDP-TCFD-technical-note.pdf'
page_range = [17, 22]


tableCDP_TCFD = ExtractPDFTables(pdf_path, page_range)
df = tableCDP_TCFD.getTablesCDP_TCFD()




In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'CDP CC 20XX - TDFD'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapCDP_TCFD()


# 10. Table Extraction and Mapping GRI 2016, 2021- GRI Oil and Gas

In [4]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI Oil and Gas 2021/GRI to GRI Oil and Gas.pdf'
page_range = list(range(4, 18))


tableFinancialReports = ExtractPDFTables(pdf_path, page_range)
df = tableFinancialReports.getTablesGRI_GRI_OILGAS_OR_COAL()

with GRI sheet would you like to map? 2016 or 2021? 2021


In [5]:
df

,STANDARD,DISCLOSURE,SECTOR \nSTANDARD \nREF #,GRI Code
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics ...,11.1.1,3-3
0,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics ...,11.2.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.3.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics ...,11.4.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.6.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.7.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.9.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.11.1,3-3
1,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics,11.12.1,3-3
2,GRI 3: Material Topics 2021,Disclosure 3-3 Management of material topics ...,11.15.1,3-3


## Mapping GRI 2016 - GRI Oil & Gas

In [ ]:
df = pd.read_csv('GRI-GRI-OIL&GAS_2016.csv')
df = df.drop('Unnamed: 0', axis=1)

path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.GRI_GRI_OIL_GAS_COAL()

## Mapping GRI 2021 - GRI Oil & Gas

In [ ]:
df = pd.read_csv('GRI-GRI-OIL&GAS_2021.csv')
df = df.drop('Unnamed: 0', axis=1)

path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2021'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.GRI_GRI_OIL_GAS_COAL()

# 11. Table Extraction and Mapping GRI Coal 2022 - GRI 2016, 2021

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI Coal 2022 - GRI/GRI to GRI Coal 2022.pdf'
page_range = list(range(2, 16))


tableFinancialReports = ExtractPDFTables(pdf_path, page_range)
df = tableFinancialReports.getTablesGRI_GRI_OILGAS_OR_COAL()

## Mapping GRI 2016 - GRI Coal

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.GRI_GRI_OIL_GAS_COAL()

## Mapping GRI 2021 - GRI Coal

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2021'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.GRI_GRI_OIL_GAS_COAL()

# 12. Table Extraction and Mapping GRI HKEX 22

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI - HKEX 2020/gri-standardslinkage-hkex-2020.pdf'
page_range = list(range(5, 9))


tableFinancialReports = ExtractPDFTables(pdf_path, page_range)
df = tableFinancialReports.getTablesGRI_HKEX20()

In [ ]:
df

## Mapping GRI 2016 - HKEX 22

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI_HKEX22()

## Mapping HKEX22 - GRI 2016

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'HKEX22'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapHKEX22_GRI()

# 13. Table Extraction and Mapping SEBI BRSB

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/SEBI-BRSB-GRI/sebi_brsb_gri_linkage_doc.pdf'
# page_range = list(range(6, 19))

tableSEBIGRI = ExtractPDFTables(pdf_path, page_range=[6, 18])
df = tableSEBIGRI.getTablesGRI_SEBI_GRI(flavor='stream')

In [ ]:
df.head(60)

## Mapping SEBI BRSB on GRI 2016 sheet

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2016'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI16_BESI()




## Mapping SEBI BRSB on GRI 2021 sheet

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet = 'GRI 2021'

mapp = MapLinks2Excel(df, sheet, path_file)
mapp.mapGRI21_BESI()




## Mapping SEBI BRSB on GRI 2018 sheet (Pending)

# 15.Table Extraction and Mapping ADX - GRI 2016

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/GRI - ADX/ADX ESG Guidance.pdf'

ADX_GRI = ExtractPDFTables(pdf_path, 
                           page_range = [10, 15],
                           flavor = 'lattice',
                           strip_text = '\n')

df = ADX_GRI.getTablesGRI_ADX()

In [ ]:
df = ADX_GRI.separate_code(df, column='GRI_Code', delimiter="\n")
# Just for mapping adx on gri16


## Mapping ADX on GRI 20016 Sheet

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet='GRI 2016'

mapp = MapLinks2Excel(df, sheet,
                      path_file,
                      ESG_to_add='ADX', 
                      first_row=3, 
                      excel_ref_column=2,
                      df_ref_column_str='GRI_Code',
                      df_ref_column_to_add='ADX_Code',
                      excel_column_to_add=25)

mapp.mapESGsRef()




## Mapping GRI on ADX sheet

In [ ]:
# df = pd.read_csv('ss.csv')
# df = df.drop('Unnamed: 0', axis=1)

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet='ADX'

mapp = MapLinks2Excel(df, sheet,
                      path_file, 
                      first_row=3,
                      excel_ref_column=1,
                      )

mapp.mapESGs()




# 16.Table Extraction and Mapping BAHRAIN BOURSE - GRI 2016

In [ ]:
pdf_path = 'ESG-Frameworks/Mapping-Standards/Bahrain Bourse - GRI16/Bahrain Bourse ESG Guide.pdf'

BAHRAIN_GRI = ExtractPDFTables(pdf_path, 
                           page_range = [1, 9],
                           flavor = 'lattice',
                           strip_text='\n')

df = BAHRAIN_GRI.getTablesGRI_BAHRAIN()


## Mapping BAHRAIN in GRI 2016 Sheet

In [ ]:
# df = pd.read_csv('ss.csv')
# df = df.drop('Unnamed: 0' ,axis=1)

In [ ]:
%%time
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet='GRI 2016'

mapp = MapLinks2Excel(df, sheet,
                      path_file,
                      ESG_to_add='BAHRAIN BOURSE', 
                      first_row=3, 
                      excel_ref_column=2,
                      df_ref_column_str='GRI_Code',
                      df_ref_column_to_add='Metric',
                      excel_column_to_add=26)

mapp.mapESGsRef()


mapp = MapLinks2Excel(df, sheet,
                      path_file,
                      ESG_to_add='BAHRAIN BOURSE', 
                      first_row=3, 
                      excel_ref_column=2,
                      df_ref_column_str='GRI_Code',
                      df_ref_column_to_add='Calculation',
                      excel_column_to_add=27)

mapp.mapESGsRef()


mapp = MapLinks2Excel(df, sheet,
                      path_file,
                      ESG_to_add='BAHRAIN BOURSE', 
                      first_row=3, 
                      excel_ref_column=2,
                      df_ref_column_str='GRI_Code',
                      df_ref_column_to_add='BAHRAIN_code',
                      excel_column_to_add=28)

mapp.mapESGsRef()




## Mapping GRI 2016 on BAHRAIN

In [ ]:
path_file = 'ESG-Frameworks/Outputs/testing.xlsx'
sheet='BAHRAIN BOURSE'

mapp = MapLinks2Excel(df, sheet,
                      path_file, 
                      first_row=3,
                      excel_ref_column=1,
                      )

mapp.mapESGs()


